In [ ]:
!pip install requests
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install requests 

In [1]:
import pandas as pd
import numpy as np

# Data Preparation

In [2]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [3]:
%%time
df = pd.read_csv("data/PS_20174392719_1491204439457_log.tar.gz", compression="gzip")

CPU times: user 20.2 s, sys: 2.23 s, total: 22.4 s
Wall time: 23.5 s


In [4]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,3,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,3,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,4,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,1,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,3,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [1]:
df

NameError: name 'df' is not defined

In [5]:
# Transform type column to be category
df['type'] = df['type'].astype('category')
df['type'] = df['type'].cat.codes

In [6]:
train = df[df['isFraud'] == 0]
test = df[df['isFraud'] == 1]

In [7]:
clean, fraudulent = train_test_split(train, test_size=0.001)
fraudulent = pd.concat([test, fraudulent]).sample(frac=1).reset_index(drop=True)
fraudulent, fraudulent_labels = fraudulent.drop(columns=['nameOrig', 'nameDest', 'isFraud', 'isFlaggedFraud'], axis=1), fraudulent['isFraud']
clean = clean.drop(columns=['nameOrig', 'nameDest', 'isFraud', 'isFlaggedFraud'], axis=1)

In [8]:
# Show clean clean data
clean.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
3764689,279,0,311498.01,922054.64,1233552.65,1783801.97,1472303.95
4035791,299,3,12003.47,0.00,0.00,0.00,0.00
2864317,227,3,18273.69,20601.00,2327.31,0.00,0.00
480902,19,3,1299.77,10384.00,9084.23,0.00,0.00
22232,8,1,40642.51,168379.00,127736.49,21266.00,69281.64


In [9]:
# Show mixed clean and fraudulent data
pd.concat([fraudulent, fraudulent_labels], axis=1).head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,396,1,118140.51,0.00,0.0,357552.30,475692.81,0
1,522,1,101408.84,101408.84,0.0,0.00,101408.84,1
2,205,2,4838.16,2977.00,0.0,1193840.00,1198678.16,0
3,249,1,49574.63,0.00,0.0,1273083.95,1322658.58,0
4,233,1,48156.69,48156.69,0.0,132064.21,180220.90,1


# Inference

In [10]:
import requests
import time 

For Autoencoder monitoring use threshold __7362628712__

In [11]:
HOST_ADDRESS = "https://demo.k8s.hydrosphere.io"

In [12]:
def send_data(data, labels=None):
    global HOST_ADDRESS, APPLICATION_NAME, SIGNATURE_NAME
    print("Using URL", f"{HOST_ADDRESS}/gateway/applications/{APPLICATION_NAME}/{SIGNATURE_NAME}")
    for index, point in data.iterrows():
        point = [point.values.tolist()]
        print(point)
        
        r = requests.post(
            url=f"{HOST_ADDRESS}/gateway/applications/{APPLICATION_NAME}/{SIGNATURE_NAME}", 
            json={'features': point})
        
        if labels is not None:
            print(r.json(), bool(labels.loc[index]))
        else:
            print(r.json())
        time.sleep(0.6)

### Random Forests

In [13]:
APPLICATION_NAME = "fraud_rf"
SIGNATURE_NAME = "infer"

In [14]:
send_data(clean)

Using URL https://demo.k8s.hydrosphere.io/gateway/applications/fraud_rf/infer
[[279.0, 0.0, 311498.01, 922054.64, 1233552.65, 1783801.97, 1472303.95]]
{'is_fraud': [False]}
[[299.0, 3.0, 12003.47, 0.0, 0.0, 0.0, 0.0]]
{'is_fraud': [False]}
[[227.0, 3.0, 18273.69, 20601.0, 2327.31, 0.0, 0.0]]
{'is_fraud': [False]}
[[19.0, 3.0, 1299.77, 10384.0, 9084.23, 0.0, 0.0]]
{'is_fraud': [False]}
[[8.0, 1.0, 40642.51, 168379.0, 127736.49, 21266.0, 69281.64]]
{'is_fraud': [False]}
[[135.0, 1.0, 185189.87, 0.0, 0.0, 778475.57, 963665.44]]
{'is_fraud': [False]}
[[12.0, 0.0, 213496.21, 30090.0, 243586.21, 0.0, 0.0]]
{'is_fraud': [False]}
[[371.0, 0.0, 39757.71, 3068607.93, 3108365.63, 4856425.78, 4816668.07]]
{'is_fraud': [False]}
[[330.0, 0.0, 170849.8, 3373169.47, 3544019.27, 1312738.52, 1141888.72]]
{'is_fraud': [False]}


KeyboardInterrupt: 

In [15]:
send_data(fraudulent, fraudulent_labels)

Using URL https://demo.k8s.hydrosphere.io/gateway/applications/fraud_rf/infer
[[396.0, 1.0, 118140.51, 0.0, 0.0, 357552.3, 475692.81]]
{'is_fraud': [False]} False
[[522.0, 1.0, 101408.84, 101408.84, 0.0, 0.0, 101408.84]]
{'is_fraud': [True]} True
[[205.0, 2.0, 4838.16, 2977.0, 0.0, 1193840.0, 1198678.16]]
{'is_fraud': [False]} False
[[249.0, 1.0, 49574.63, 0.0, 0.0, 1273083.95, 1322658.58]]
{'is_fraud': [False]} False
[[233.0, 1.0, 48156.69, 48156.69, 0.0, 132064.21, 180220.9]]
{'is_fraud': [True]} True
[[328.0, 1.0, 2299330.46, 2299330.46, 0.0, 1403092.21, 3736457.5]]
{'is_fraud': [True]} True
[[264.0, 1.0, 2514878.33, 2514878.33, 0.0, 1017845.77, 3532724.1]]
{'is_fraud': [True]} True
[[155.0, 3.0, 12183.08, 0.0, 0.0, 0.0, 0.0]]
{'is_fraud': [False]} False
[[636.0, 1.0, 222517.07, 222517.07, 0.0, 130580.85, 353097.93]]
{'is_fraud': [True]} True
[[157.0, 1.0, 421925.04, 11338.0, 0.0, 4771281.0, 5193206.04]]
{'is_fraud': [False]} False
[[712.0, 1.0, 135977.23, 135977.23, 0.0, 314329.11,

KeyboardInterrupt: 